<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise



In [11]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install requests
!pip install matplotlib
!pip install seaborn
!pip install plotly
!pip install pandas
!pip install pyinputplus
!pip install fetch-my-weather
!pip install hands-on-ai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=c36c74edf24173f74296e023231d5d4129e9a7801e1835e5eb79ff744537b63f
  Stored in directory: /root/.cache/pip/wheels/a3/6b/89/3bec14288af8e0729d088fb41a785e583d49e772f2529c7676
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16224 sha256=76f6ee7f680a4af135fa73e9344afb6ba4865955d4696b40120be68441fa3970
  Stored in directory: /root/.cache/pip/wheels/fc/40/7a/4d3b2dc2e80b4b3bbda89eec94fcefd6c8bd1101cc7bb89554
  Created wheel for stdiomask: filename=stdiomask

In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')
#7cf20335110caaf78db0fecb31852d45

## 📦 Setup and Configuration
Import required packages and setup environment.

In [1]:
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd
import re

# I added two more codes to enhance the visualisation
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response
# Typing support for cleaner, testable function signatures
from typing import Union, List, Dict

ModuleNotFoundError: No module named 'fetch_my_weather'

## 🌤️ Weather Data Functions

In [2]:
# Define get_weather_data() function here
from typing import Union, List, Dict

def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    Fetch weather data for one or more locations and return it as a dictionary.

    Args:
        location (str or list): A single location name or a list of locations.
        forecast_days (int): Number of forecast days to retrieve (default is 5).

    Returns:
        dict: A dictionary mapping each location to its weather data or error.
              Format:
              {
                  "Paris": {
                      "current_conditions": {...},
                      "forecast": [{...}, {...}, ...]
                  },
                  "InvalidCity": {
                      "error": "Could not retrieve weather data"
                  }
              }
    """
    # If a single location string is provided, convert it to a list for uniform processing
    if isinstance(location, str):
        location = [location]  # Convert single string to list

    results = {} # Initialize dictionary to store weather data for each location

    # Loop through each location to fetch and process weather data
    for loc in location:
        try:                                                    # Call the assumed external function get_weather to retrieve weather info for the location
            weather_response = get_weather(loc)                 # Verify the returned object supports conversion to dictionary format
            if not hasattr(weather_response, 'dict'):
                raise TypeError("Weather object does not support dict() conversion.")

            # Convert the weather response object to a dictionary for easier data access
            weather_data = weather_response.dict()

            # Extract forecast and current conditions safely using .get() with default empty values
            forecast = weather_data.get("forecast", [])
            current = weather_data.get("current_conditions", {})

            # Raise an error if forecast data is missing or empty, to handle incomplete data cases
            if not forecast:
                raise ValueError("Forecast data is missing or empty.")

            #  Store the current conditions and limited forecast data (up to forecast_days) in the results
            results[loc] = {
                "current_conditions": current or "No current data available",
                "forecast": forecast[:forecast_days]
            }
        # If any error occurs (e.g., network, parsing, missing data), record the error message for the location
        except Exception as e:
            results[loc] = {"error": f"Failed to retrieve weather for {loc}: {e}"}
    # Return the compiled dictionary containing weather data or errors for all requested locations
    return results



## 📊 Visualisation Functions

In [4]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
import plotly.express as px
import pandas as pd

def create_temperature_visualisation_interactive(weather_data):
    """
    Create an interactive temperature line chart using Plotly.
    """
    # Flatten the nested weather_data dictionary into a list of records
    rows = []
    for location, times_data in weather_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,               # Store city/location name
                "Time": time,                       # Store time period (e.g., date or hour)
                "Temperature": info["temperature"]  # Store temperature value
            })

    # Convert the list of records into a pandas DataFrame for easy plotting
    df = pd.DataFrame(rows)

    # Create an interactive line chart:
    # - x axis: Time periods
    # - y axis: Temperature values
    # - color: separate lines by Location
    # - markers: show points on the line
    # - title and axis labels for clarity

    fig = px.line(df, x="Time", y="Temperature", color="Location",
                  markers=True, title="Interactive Temperature Forecast",
                  labels={"Temperature": "Temp (°C)", "Time": "Time Period"})

    # Customize hover info to show temperature with °C, time, and location name
    fig.update_traces(mode='lines+markers', hovertemplate="%{y}°C at %{x}<br>%{fullData.name}")
    # Rotate x-axis labels for better readability
    fig.update_layout(xaxis_tickangle=-45)
     # Display the interactive plot in the output
    fig.show()





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def create_precipitation_visualisation_from_df(df: pd.DataFrame, output_type='display'):
    """
    Create a grouped bar chart of precipitation chances using a pandas DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with columns ['Location', 'Time', 'Precipitation']
        output_type (str): 'display' to show the plot, 'figure' to return the figure

    Returns:
        matplotlib.figure.Figure or None
    """

    # Use seaborn-darkgrid style for better visual appearance
    plt.style.use('seaborn-darkgrid')
    # Create a figure and axis object with specified size
    fig, ax = plt.subplots(figsize=(10, 6))

    # Get unique locations and time periods from the DataFrame
    locations = df['Location'].unique()
    time_labels = df['Time'].unique()
    # Calculate bar width based on number of locations to fit bars side-by-side
    width = 0.8 / len(locations)
    x = range(len(time_labels))

    # Loop through each location to plot its precipitation bars
    for idx, loc in enumerate(locations):
        # Filtering data for the current location
        loc_data = df[df['Location'] == loc]
        loc_data = loc_data.set_index('Time').reindex(time_labels).reset_index()
         # Calculate bar positions with offset for each location to avoid overlap
        offsets = [i + (idx * width) for i in x]
         # Extract precipitation values, fill missing values with 0
        precip = loc_data['Precipitation'].fillna(0).tolist()

        ax.bar(offsets, precip, width=width, label=loc)
        # Annotate each bar with its precipitation value (percentage)
        for i, value in enumerate(precip):
            ax.text(offsets[i], value + 1, f"{value}%", ha='center', fontsize=8)

    # Set chart title and axis labels
    ax.set(
        title="Chance of Rain by Location",
        xlabel="Time Period",
        ylabel="Chance of Rain (%)"
    )
    ax.set_xticks([i + width * (len(locations)-1)/2 for i in x])
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(True)

    # Return the figure if requested, otherwise display the plot
    if output_type == 'figure':
        return fig
    plt.show()




## 🤖 Natural Language Processing

In [13]:
def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    # Extract parsed query components: location, time period, and weather attribute
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']

    # Check if forecast data exists for the requested location and time
    if loc not in data or time not in data[loc]:
        return f"❌ No forecast available for {loc} {time}."

    # Retrieve the forecast dictionary for the given location and time
    forecast = data[loc][time]

    # Define helper functions to convert temperature and wind speed to desired units
    # These assume you have functions convert_temperature() and convert_wind_speed() defined elsewhere
    def convert_temp(c): return f"{convert_temperature(c, temp_unit)}°{'F' if temp_unit == 'Fahrenheit' else 'C'}"
    # Convert wind speed to desired unit and append unit label
    def convert_wind(w): return f"{convert_wind_speed(w, wind_unit)} {wind_unit}"

    # Map weather attributes to corresponding response-generating lambda functions
    # Each lambda returns a formatted string including emoji, location, time, and converted values
    responses = {
        'temperature': lambda: f"🌡️ Temperature in {loc} {time} is {convert_temp(forecast.get('temperature', 'unknown'))}.",
        'rain': lambda: f"🌧️ Expect {'rain' if forecast.get('rain') else 'no rain'} in {loc} {time}.",
        'snow': lambda: f"❄️ Expect {'snow' if forecast.get('snow') else 'no snow'} in {loc} {time}.",
        'wind': lambda: f"💨 Wind speed in {loc} {time} is {convert_wind(forecast.get('wind', 'unknown'))}.",
        'humidity': lambda: f"💧 Humidity in {loc} {time} is {forecast.get('humidity', 'unknown')}%."
    }

    # If the requested attribute is recognized, call the corresponding lambda to get the response
    if attr in responses:
        return responses[attr]()
    else:
        # For 'general' or unrecognised attributes, provide a default summary response
        temp = convert_temp(forecast.get('temperature', 'unknown'))
        rain = 'rain' if forecast.get('rain') else 'no rain'
        return f"In {loc} {time}, temperature is {temp} with {rain}."


## 🧭 User Interface

In [14]:
# Define menu functions using pyinputplus or ipywidgets here

import pyinputplus as pyip

# Print a banner to welcome the user to the Weather Assistant
def print_banner():
    print("\n" + "="*40)
    print("☁️  Welcome to the Weather Assistant!  ☁️")
    print("="*40)

# Display the main menu and prompt the user to select an option using pyinputplus inputMenu
def main_menu():
    print("\n=== 🌦️ Weather Assistant Menu ===")
    return pyip.inputMenu(
        ["🌤️  Ask Weather Question", "📊  Visualize Weather Data", "🚪  Exit"],
        lettered=True,
        numbered=False,
        prompt="\n🌈 What would you like to do?\n",
        blank=False,
        limit=3,
        retries=2,
        default="🚪  Exit"
    )

# Display the visualization submenu to choose between temperature, precipitation, or going back
    # Uses similar pyinputplus inputMenu settings as main_menu
def visualization_menu():
    return pyip.inputMenu(
        ["📈  Temperature", "🌧️  Precipitation", "🔙  Back to Main Menu"],
        lettered=True,
        numbered=False,
        prompt="\n📊 Choose a visualization type:\n",
        blank=False,
        limit=3,
        retries=2,
        default="🔙  Back to Main Menu"
    )

 # Main loop to run the menu-driven interface
def run_interface():
    print_banner() # Show welcome banner once at start
    while True:
        choice = main_menu() # Show the main menu and get the user choice

        if "Ask Weather" in choice: # User selected to ask a weather question
            print("🗣️  You chose to ask a weather question.") #call the function that handles weather questions

        elif "Visualize" in choice:  # User selected to visualize weather data
            while True:
                sub_choice = visualization_menu() # Show visualization submenu
                if "Temperature" in sub_choice:
                    print("📈 Showing temperature chart...")  # Call temperature visualization function here
                elif "Precipitation" in sub_choice:
                    print("🌧️ Showing precipitation chart...") # Call precipitation visualization function here
                elif "Back" in sub_choice:
                    break   # User wants to go back to main menu, break inner loop

        elif "Exit" in choice:
            print("👋 Exiting Weather Assistant. Stay safe!")
            break # Exit the main loop and end program



## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
#-------------------------------------------------------------------------------------------------------
# 1.🌤️ Weather Data Functions
#-------------------------------------------------------------------------------------------------------
from typing import Union, List, Dict

def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    Fetch weather data for one or more locations and return it as a dictionary.

    Args:
        location (str or list): A single location name or a list of locations.
        forecast_days (int): Number of forecast days to retrieve (default is 5).

    Returns:
        dict: A dictionary mapping each location to its weather data or error.
              Format:
              {
                  "Paris": {
                      "current_conditions": {...},
                      "forecast": [{...}, {...}, ...]
                  },
                  "InvalidCity": {
                      "error": "Could not retrieve weather data"
                  }
              }
    """
    if isinstance(location, str):
        location = [location]

    results = {}

    for loc in location:
        try:
            weather_response = get_weather(loc)
            if not hasattr(weather_response, 'dict'):
                raise TypeError("Weather object does not support dict() conversion.")

            weather_data = weather_response.dict()
            forecast = weather_data.get("forecast", [])
            current = weather_data.get("current_conditions", {})


            if not forecast:
                raise ValueError("Forecast data is missing or empty.")


            results[loc] = {
                "current_conditions": current or "No current data available",
                "forecast": forecast[:forecast_days]
            }

        except Exception as e:
            results[loc] = {"error": f"Failed to retrieve weather for {loc}: {e}"}
    return results

#-------------------------------------------------------------------------------------------------------
# 2. 📊 Visualisation Functions
#-------------------------------------------------------------------------------------------------------
#def create_temperature_visualisation

import plotly.express as px
import pandas as pd

def create_temperature_visualisation_interactive(weather_data):
    """
    Create an interactive temperature line chart using Plotly.
    """
    rows = []
    for location, times_data in weather_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })


    df = pd.DataFrame(rows)

    fig = px.line(df, x="Time", y="Temperature", color="Location",
                  markers=True, title="Interactive Temperature Forecast",
                  labels={"Temperature": "Temp (°C)", "Time": "Time Period"})

    fig.update_traces(mode='lines+markers', hovertemplate="%{y}°C at %{x}<br>%{fullData.name}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()
import pandas as pd
import matplotlib.pyplot as plt

#create_precipitation_visualisation

def create_precipitation_visualisation_from_df(df: pd.DataFrame, output_type='display'):
    """
    Create a grouped bar chart of precipitation chances using a pandas DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with columns ['Location', 'Time', 'Precipitation']
        output_type (str): 'display' to show the plot, 'figure' to return the figure

    Returns:
        matplotlib.figure.Figure or None
    """

    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    locations = df['Location'].unique()
    time_labels = df['Time'].unique()
    width = 0.8 / len(locations)
    x = range(len(time_labels))

    for idx, loc in enumerate(locations):

        loc_data = df[df['Location'] == loc]
        loc_data = loc_data.set_index('Time').reindex(time_labels).reset_index()

        offsets = [i + (idx * width) for i in x]

        precip = loc_data['Precipitation'].fillna(0).tolist()

        ax.bar(offsets, precip, width=width, label=loc)
        for i, value in enumerate(precip):
            ax.text(offsets[i], value + 1, f"{value}%", ha='center', fontsize=8)

    ax.set(
        title="Chance of Rain by Location",
        xlabel="Time Period",
        ylabel="Chance of Rain (%)"
    )
    ax.set_xticks([i + width * (len(locations)-1)/2 for i in x])
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(True)

    if output_type == 'figure':
        return fig
    plt.show()


#-------------------------------------------------------------------------------------------------------
# 3. 🤖 Natural Language Processing
#-------------------------------------------------------------------------------------------------------


def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']

    if loc not in data or time not in data[loc]:
        return f"❌ No forecast available for {loc} {time}."

    forecast = data[loc][time]

    def convert_temp(c): return f"{convert_temperature(c, temp_unit)}°{'F' if temp_unit == 'Fahrenheit' else 'C'}"
    def convert_wind(w): return f"{convert_wind_speed(w, wind_unit)} {wind_unit}"

    responses = {
        'temperature': lambda: f"🌡️ Temperature in {loc} {time} is {convert_temp(forecast.get('temperature', 'unknown'))}.",
        'rain': lambda: f"🌧️ Expect {'rain' if forecast.get('rain') else 'no rain'} in {loc} {time}.",
        'snow': lambda: f"❄️ Expect {'snow' if forecast.get('snow') else 'no snow'} in {loc} {time}.",
        'wind': lambda: f"💨 Wind speed in {loc} {time} is {convert_wind(forecast.get('wind', 'unknown'))}.",
        'humidity': lambda: f"💧 Humidity in {loc} {time} is {forecast.get('humidity', 'unknown')}%."
    }

    if attr in responses:
        return responses[attr]()
    else:
        temp = convert_temp(forecast.get('temperature', 'unknown'))
        rain = 'rain' if forecast.get('rain') else 'no rain'
        return f"In {loc} {time}, temperature is {temp} with {rain}."

#-------------------------------------------------------------------------------------------------------
# 4. 🧭 User Interface
#-------------------------------------------------------------------------------------------------------

import pyinputplus as pyip

def print_banner():
    print("\n" + "="*40)
    print("☁️  Welcome to the Weather Assistant!  ☁️")
    print("="*40)


def main_menu():
    print("\n=== 🌦️ Weather Assistant Menu ===")
    return pyip.inputMenu(
        ["🌤️  Ask Weather Question", "📊  Visualize Weather Data", "🚪  Exit"],
        lettered=True,
        numbered=False,
        prompt="\n🌈 What would you like to do?\n",
        blank=False,
        limit=3,
        retries=2,
        default="🚪  Exit"
    )


def visualization_menu():
    return pyip.inputMenu(
        ["📈  Temperature", "🌧️  Precipitation", "🔙  Back to Main Menu"],
        lettered=True,
        numbered=False,
        prompt="\n📊 Choose a visualization type:\n",
        blank=False,
        limit=3,
        retries=2,
        default="🔙  Back to Main Menu"
    )


def run_interface():
    print_banner()
    while True:
        choice = main_menu()

        if "Ask Weather" in choice:
            print("🗣️  You chose to ask a weather question.")

        elif "Visualize" in choice:
            while True:
                sub_choice = visualization_menu()
                if "Temperature" in sub_choice:
                    print("📈 Showing temperature chart...")
                elif "Precipitation" in sub_choice:
                    print("🌧️ Showing precipitation chart...")
                elif "Back" in sub_choice:
                    break

        elif "Exit" in choice:
            print("👋 Exiting Weather Assistant. Stay safe!")
            break


## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

#-------------------------------------------------------------------------------------------------------
# 1.🌤️ Weather Data Functions
#-------------------------------------------------------------------------------------------------------

# 1.1 ✅ Define the get_weather_data() Function
from typing import Union, List, Dict

def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    Fetch weather data for one or more locations and return it as a dictionary.

    Args:
        location (str or list): A single location name or a list of locations.
        forecast_days (int): Number of forecast days to retrieve (default is 5).

    Returns:
        dict: A dictionary mapping each location to its weather data or error.
    """
    if isinstance(location, str):
        location = [location]

    results = {}

    for loc in location:
        try:
            weather_response = get_weather(loc)
            if not hasattr(weather_response, 'dict'):
                raise TypeError("Weather object does not support dict() conversion.")

            weather_data = weather_response.dict()
            forecast = weather_data.get("forecast", [])
            current = weather_data.get("current_conditions", {})

            if not forecast:
                raise ValueError("Forecast data is missing or empty.")

            results[loc] = {
                "current_conditions": current or "No current data available",
                "forecast": forecast[:forecast_days]
            }

        except Exception as e:
            results[loc] = {"error": f"Failed to retrieve weather for {loc}: {e}"}

    return results

# 1.2 ✅ Run doctest

import doctest
doctest.testmod()


def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    >>> isinstance(get_weather_data("Paris"), dict)
    True
    >>> "error" in get_weather_data("FakeCity")["FakeCity"]
    True
    """

# 1.3 ✅ Run unittest

import unittest

class MockWeatherResponse:
    def __init__(self, valid=True):
        self.valid = valid

    def dict(self):
        if self.valid:
            return {
                "current_conditions": {"temperature": 20},
                "forecast": [{"day": 1}, {"day": 2}, {"day": 3}]
            }
        else:
            return {}

def get_weather(location):
    if location == "FakeCity":
        raise ValueError("Location not found")
    return MockWeatherResponse()

class TestGetWeatherData(unittest.TestCase):
    def test_valid_city(self):
        result = get_weather_data("Paris", forecast_days=2)
        self.assertIn("current_conditions", result["Paris"])
        self.assertEqual(len(result["Paris"]["forecast"]), 2)

    def test_invalid_city(self):
        result = get_weather_data("FakeCity")
        self.assertIn("error", result["FakeCity"])

unittest.main(argv=[''], verbosity=2, exit=False)

# 1.4 🐞 Debugging

def debug_locations(locations):
    print("🌍 Fetching weather for multiple locations...")
    results = get_weather_data(locations)
    for loc, data in results.items():
        print(f"\n📍 Location: {loc}")
        if "error" in data:
            print(f"❌ Error: {data['error']}")
        else:
            print(f"✅ Current: {data['current_conditions']}")
            print(f"📅 Forecast ({len(data['forecast'])} days):")
            for day in data['forecast']:
                print(day)

debug_locations(["London", "FakeCity", "Berlin"])

#-------------------------------------------------------------------------------------------------------
# 2. 📊 Visualisation Functions
#-------------------------------------------------------------------------------------------------------

# 2.1 Define the Visualisation Functions

# 2.1.1 Temperature Line Chart (Plotly)

import plotly.express as px
import pandas as pd

def create_temperature_visualisation_interactive(weather_data):
    """
    Create an interactive temperature line chart using Plotly.
    """
    rows = []
    for location, times_data in weather_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })

    df = pd.DataFrame(rows)
    fig = px.line(
        df, x="Time", y="Temperature", color="Location",
        markers=True, title="Interactive Temperature Forecast",
        labels={"Temperature": "Temp (°C)", "Time": "Time Period"}
    )
    fig.update_traces(mode='lines+markers', hovertemplate="%{y}°C at %{x}<br>%{fullData.name}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

# 2.1.2 Precipitation Bar Chart (Matplotlib)

import pandas as pd
import matplotlib.pyplot as plt

def create_precipitation_visualisation_from_df(df: pd.DataFrame, output_type='display'):
    """
    Create a grouped bar chart of precipitation chances using a pandas DataFrame.
    """
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    locations = df['Location'].unique()
    time_labels = df['Time'].unique()
    width = 0.8 / len(locations)
    x = range(len(time_labels))

    for idx, loc in enumerate(locations):
        loc_data = df[df['Location'] == loc]
        loc_data = loc_data.set_index('Time').reindex(time_labels).reset_index()
        offsets = [i + (idx * width) for i in x]
        precip = loc_data['Precipitation'].fillna(0).tolist()

        ax.bar(offsets, precip, width=width, label=loc)
        for i, value in enumerate(precip):
            ax.text(offsets[i], value + 1, f"{value}%", ha='center', fontsize=8)

    ax.set(
        title="Chance of Rain by Location",
        xlabel="Time Period",
        ylabel="Chance of Rain (%)"
    )
    ax.set_xticks([i + width * (len(locations)-1)/2 for i in x])
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(True)

    if output_type == 'figure':
        return fig
    plt.show()

# 2.2 ✅ Run doctest for Format Check

import doctest
doctest.testmod()


def create_temperature_visualisation_interactive(weather_data):
    """
    >>> isinstance(weather_data, dict)
    True
    """
    ...

# 2.3 ✅ Run unittest for Chart Functions

import unittest

class TestVisualisationData(unittest.TestCase):
    def test_dataframe_conversion(self):
        data = {
            "London": {"Mon": {"temperature": 21}, "Tue": {"temperature": 22}},
            "Paris": {"Mon": {"temperature": 19}, "Tue": {"temperature": 20}},
        }
        rows = []
        for location, times_data in data.items():
            for time, info in times_data.items():
                rows.append({
                    "Location": location,
                    "Time": time,
                    "Temperature": info["temperature"]
                })
        df = pd.DataFrame(rows)
        self.assertEqual(len(df), 4)
        self.assertIn("Location", df.columns)

unittest.main(argv=[''], verbosity=2, exit=False)

# 2.4 ✅ Run assert Tests (Pytest Style)

sample_data = {
    "London": {"Monday": {"temperature": 20}, "Tuesday": {"temperature": 22}},
    "Paris": {"Monday": {"temperature": 18}, "Tuesday": {"temperature": 21}}
}

def test_temperature_data():
    rows = []
    for location, times_data in sample_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })
    df = pd.DataFrame(rows)
    assert df.shape[0] == 4
    assert "Temperature" in df.columns

test_temperature_data()
print("All visualisation data tests passed.")

# 2.5 🐞 Debug with Sample Charts

# Temperature (Interactive)
create_temperature_visualisation_interactive({
    "London": {"Monday": {"temperature": 20}, "Tuesday": {"temperature": 22}},
    "Paris": {"Monday": {"temperature": 18}, "Tuesday": {"temperature": 21}}
})

# Precipitation (Grouped Bar)
df = pd.DataFrame({
    "Location": ["London", "Paris", "London", "Paris"],
    "Time": ["Monday", "Monday", "Tuesday", "Tuesday"],
    "Precipitation": [30, 50, 10, 60]
})
create_precipitation_visualisation_from_df(df)

#-------------------------------------------------------------------------------------------------------
# 3. 🤖 Natural Language Processing
#-------------------------------------------------------------------------------------------------------

# 3.1 ✅ Define generate_weather_response()

def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']

    if loc not in data or time not in data[loc]:
        return f"❌ No forecast available for {loc} {time}."

    forecast = data[loc][time]

    def convert_temp(c): return f"{convert_temperature(c, temp_unit)}°{'F' if temp_unit == 'Fahrenheit' else 'C'}"
    def convert_wind(w): return f"{convert_wind_speed(w, wind_unit)} {wind_unit}"

    responses = {
        'temperature': lambda: f"🌡️ Temperature in {loc} {time} is {convert_temp(forecast.get('temperature', 'unknown'))}.",
        'rain': lambda: f"🌧️ Expect {'rain' if forecast.get('rain') else 'no rain'} in {loc} {time}.",
        'snow': lambda: f"❄️ Expect {'snow' if forecast.get('snow') else 'no snow'} in {loc} {time}.",
        'wind': lambda: f"💨 Wind speed in {loc} {time} is {convert_wind(forecast.get('wind', 'unknown'))}.",
        'humidity': lambda: f"💧 Humidity in {loc} {time} is {forecast.get('humidity', 'unknown')}%."
    }

    if attr in responses:
        return responses[attr]()
    else:
        temp = convert_temp(forecast.get('temperature', 'unknown'))
        rain = 'rain' if forecast.get('rain') else 'no rain'
        return f"In {loc} {time}, temperature is {temp} with {rain}."

# 3.2 ✅ Add doctest

def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    """
    >>> parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'temperature'}
    >>> data = {'Paris': {'today': {'temperature': 20}}}
    >>> generate_weather_response(parsed, data)
    '🌡️ Temperature in Paris today is 20°C.'
    """

import doctest
doctest.testmod()

# 3.3 ✅ Unit Testing with unittest

# Mock functions for temperature/wind conversion:

def convert_temperature(value, unit):
    return round((value * 9/5 + 32) if unit == 'Fahrenheit' else value)

def convert_wind_speed(value, unit):
    return round((value / 1.609) if unit == 'mph' else value)

# Then test:
import unittest

class TestGenerateWeatherResponse(unittest.TestCase):
    def test_temperature_celsius(self):
        parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'temperature'}
        data = {'Paris': {'today': {'temperature': 25}}}
        result = generate_weather_response(parsed, data)
        self.assertIn("25°C", result)

    def test_missing_location(self):
        parsed = {'location': 'Mars', 'time_period': 'today', 'attribute': 'temperature'}
        data = {'Paris': {'today': {'temperature': 25}}}
        result = generate_weather_response(parsed, data)
        self.assertIn("❌", result)

unittest.main(argv=[''], verbosity=2, exit=False)

# 3.4 ✅ Use assert (Pytest-style tests)

parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'wind'}
data = {'Paris': {'today': {'wind': 20}}}

def test_weather_response():
    result = generate_weather_response(parsed, data, wind_unit='km/h')
    assert "20 km/h" in result
    assert "💨" in result

test_weather_response()
print("All pytest-style tests passed.")

# 3.5 🐞 Debug with Sample Data

parsed_inputs = [
    {'location': 'London', 'time_period': 'today', 'attribute': 'temperature'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'rain'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'wind'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'snow'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'general'},
]

weather_data = {
    "London": {
        "today": {
            "temperature": 22,
            "rain": False,
            "snow": False,
            "wind": 15,
            "humidity": 70
        }
    }
}

for parsed in parsed_inputs:
    print(generate_weather_response(parsed, weather_data))

#-------------------------------------------------------------------------------------------------------
# 4. 🧭 User Interface
#-------------------------------------------------------------------------------------------------------
# 4.1 ✅ Define the Menu Interface Functions

import pyinputplus as pyip

def print_banner():
    print("\n" + "="*40)
    print("☁️  Welcome to the Weather Assistant!  ☁️")
    print("="*40)

def main_menu():
    print("\n=== 🌦️ Weather Assistant Menu ===")
    return pyip.inputMenu(
        ["🌤️  Ask Weather Question", "📊  Visualize Weather Data", "🚪  Exit"],
        lettered=True,
        numbered=False,
        prompt="\n🌈 What would you like to do?\n",
        blank=False,
        limit=3,
        retries=2,
        default="🚪  Exit"
    )

def visualization_menu():
    return pyip.inputMenu(
        ["📈  Temperature", "🌧️  Precipitation", "🔙  Back to Main Menu"],
        lettered=True,
        numbered=False,
        prompt="\n📊 Choose a visualization type:\n",
        blank=False,
        limit=3,
        retries=2,
        default="🔙  Back to Main Menu"
    )

def run_interface():
    print_banner()
    while True:
        choice = main_menu()
        if "Ask Weather" in choice:
            print("🗣️  You chose to ask a weather question.")
        elif "Visualize" in choice:
            while True:
                sub_choice = visualization_menu()
                if "Temperature" in sub_choice:
                    print("📈 Showing temperature chart...")
                elif "Precipitation" in sub_choice:
                    print("🌧️ Showing precipitation chart...")
                elif "Back" in sub_choice:
                    break
        elif "Exit" in choice:
            print("👋 Exiting Weather Assistant. Stay safe!")
            break

# 4.2 ✅ Add Doctests

def main_menu():
    """
    >>> isinstance(main_menu(), str)
    True
    """

import doctest
doctest.testmod()

# 4.3 ✅ Unittest for Menu Logic

import unittest

class TestMenuStructure(unittest.TestCase):
    def test_main_menu_options(self):
        options = [
            "🌤️  Ask Weather Question",
            "📊  Visualize Weather Data",
            "🚪  Exit"
        ]
        self.assertEqual(len(options), 3)
        self.assertIn("🌤️  Ask Weather Question", options)

unittest.main(argv=[''], verbosity=2, exit=False)


# 4.4 ✅ Assert-Based Tests (Pytest-style)

def test_menu_choice_return_type():
    menu_response = "🌧️  Precipitation"
    assert isinstance(menu_response, str)
    assert "Precipitation" in menu_response

test_menu_choice_return_type()
print("✅ Menu tests passed.")

# 4.5 🐞 Debug with Manual Simulation

# Run this in a local terminal or Colab (with ipywidgets disabled):
run_interface()

# Expected steps:
# - Shows banner
# - Main menu with 3 choices
# - If 'Visualize Weather Data' is chosen:
#     - Shows 3 submenu options
#     - Returns to main menu on 'Back'


